# Simple Census Grab

### Load a few packages.

In [ ]:
import os

import geopandas as gpd
import pandas as pd
import requests
import folium

%matplotlib inline

### Get poverty data from the census.
See the possible variables, here:
http://api.census.gov/data/2015/acs5/variables.html

In [ ]:
acs  = "https://api.census.gov/data/2015/acs5/profile?"
# resp = requests.get(acs + "get=NAME,DP03_0119PE&for=tract:*&in=state:17+county:*")
json = resp.json()

df = gpd.pd.DataFrame(data = json[1:], columns = ["Name", "DP03_0119PE", "state", "county", "tract"])
df["Poverty Rate"] = pd.to_numeric(df["DP03_0119PE"], errors = "coerce").fillna(0)

### Get shapefiles of census tracts.

In [ ]:
# os.system("wget http://www2.census.gov/geo/tiger/GENZ2014/shp/cb_2014_17_tract_500k.zip")
# os.system("unzip cb_2014_17_tract_500k.zip")
tract_df = gpd.read_file("cb_2014_17_tract_500k.shp")

### Merge them together.

In [ ]:
merged_df = tract_df.merge(df, left_on  = ["STATEFP", "COUNTYFP", "TRACTCE"], 
                               right_on = ["state", "county", "tract"])

### See it.

In [ ]:
m = folium.Map([41.9901466,-87.6620448], zoom_start = 13, tiles = "cartodbpositron", max_zoom = 14, min_zoom = 6, attr = "")

colormap = folium.LinearColormap(("orange", "white", "purple"), vmin = 70, vmax = 100, caption = "Percent of Families Poverty")
colormap.add_to(m)

folium.GeoJson(merged_df,
               style_function = lambda feature: { 
                  'fillColor': colormap(feature['properties']["Poverty Rate"]) if feature["properties"]["Poverty Rate"] else "k",
                  "color" : "k", "weight" : 0.3, "fillOpacity" : 0.4 if feature["properties"]["Poverty Rate"] else 0,
               }).add_to(m)
m

In [ ]:
m.save("il_poverty.html")

In [ ]:
! scp il_poverty.html saxon.harris.uchicago.edu:www/